# 🎥 Comaneci Video Generator

This notebook will:
1. Upload your images to Google Drive
2. Generate videos using Replicate
3. Save videos back to your Google Drive
4. Run without your computer on!

## Setup Instructions
1. Run the first cell to install dependencies
2. Enter your Replicate API key
3. Connect to Google Drive
4. Upload your images
5. Hit 'Run All' and close the tab!


In [ ]:
# Install required packages
!pip install replicate google-auth-oauthlib google-auth-httplib2 google-api-python-client

In [ ]:
# Import required libraries
import os
import replicate
import base64
from google.colab import drive
from google.colab import files
import time
from pathlib import Path
import requests
from IPython.display import HTML, display
import json

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Set your Replicate API token
REPLICATE_API_TOKEN = input("Enter your Replicate API token: ")
os.environ['REPLICATE_API_TOKEN'] = REPLICATE_API_TOKEN
replicate_client = replicate.Client(api_token=REPLICATE_API_TOKEN)

# Test the connection
try:
    response = requests.get(
        'https://api.replicate.com/v1/models/kwaivgi/kling-v1.6-standard',
        headers={'Authorization': f'Token {REPLICATE_API_TOKEN}'}
    )
    if response.status_code == 200:
        print("✅ Successfully connected to Replicate API")
    else:
        print(f"❌ Error connecting to Replicate: {response.status_code} - {response.text}")
except Exception as e:
    print(f"❌ Error connecting to Replicate: {str(e)}")

In [ ]:
# Create folders in Google Drive
DRIVE_INPUT_FOLDER = '/content/drive/MyDrive/Comaneci/input'
DRIVE_OUTPUT_FOLDER = '/content/drive/MyDrive/Comaneci/output'

# Create folders if they don't exist
os.makedirs(DRIVE_INPUT_FOLDER, exist_ok=True)
os.makedirs(DRIVE_OUTPUT_FOLDER, exist_ok=True)

print("✅ Created folders in Google Drive:")
print(f"   📁 Input: {DRIVE_INPUT_FOLDER}")
print(f"   📁 Output: {DRIVE_OUTPUT_FOLDER}")

In [ ]:
def process_image(image_path, prompt="", aspect_ratio="16:9", duration=5):
    """Process a single image and return the video URL"""
    try:
        print(f"\n🎬 Processing: {image_path}")
        
        # Read and encode image
        with open(image_path, 'rb') as f:
            image_bytes = f.read()
            image_b64 = base64.b64encode(image_bytes).decode('utf-8')
            
            # Get correct MIME type
            ext = Path(image_path).suffix.lower()
            mime_type = {
                '.jpg': 'image/jpeg',
                '.jpeg': 'image/jpeg',
                '.png': 'image/png',
                '.webp': 'image/webp',
                '.gif': 'image/gif'
            }.get(ext, 'image/jpeg')
            
            image_uri = f"data:{mime_type};base64,{image_b64}"
        
        # Generate video
        prediction = replicate.run(
            "kwaivgi/kling-v1.6-standard",
            input={
                "start_image": image_uri,
                "prompt": prompt,
                "aspect_ratio": aspect_ratio,
                "duration": int(duration),
                "cfg_scale": 0.5,
                "negative_prompt": ""
            }
        )
        
        if prediction:
            print(f"✅ Video generated successfully")
            return prediction
        else:
            print(f"❌ No output received from API")
            return None
            
    except Exception as e:
        print(f"❌ Error processing image: {str(e)}")
        return None

def download_video(url, output_path):
    """Download video from URL to output path"""
    try:
        print(f"📥 Downloading video to: {output_path}")
        response = requests.get(url, stream=True)
        response.raise_for_status()
        
        with open(output_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
                    
        print("✅ Video downloaded successfully")
        return True
    except Exception as e:
        print(f"❌ Error downloading video: {str(e)}")
        return False

In [ ]:
# Settings for video generation
prompt = input("Enter prompt (or leave empty): ")
aspect_ratio = input("Enter aspect ratio (16:9, 9:16, or 1:1) [default: 16:9]: ") or "16:9"
duration = int(input("Enter duration in seconds [default: 5]: ") or "5")

print("\n⚙️ Settings:")
print(f"   🗣️ Prompt: {prompt}")
print(f"   📐 Aspect Ratio: {aspect_ratio}")
print(f"   ⏱️ Duration: {duration} seconds")

In [ ]:
# Process all images in the input folder
results = []
processed = 0
failed = 0

# Get all image files
image_files = [
    f for f in os.listdir(DRIVE_INPUT_FOLDER)
    if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.webp'))
]

print(f"\n🎯 Found {len(image_files)} images to process\n")

for image_file in image_files:
    image_path = os.path.join(DRIVE_INPUT_FOLDER, image_file)
    output_name = f"{Path(image_file).stem}_video.mp4"
    output_path = os.path.join(DRIVE_OUTPUT_FOLDER, output_name)
    
    # Generate and download video
    video_url = process_image(image_path, prompt, aspect_ratio, duration)
    
    if video_url and download_video(video_url, output_path):
        processed += 1
        results.append({
            'image': image_file,
            'video': output_name,
            'status': 'success'
        })
    else:
        failed += 1
        results.append({
            'image': image_file,
            'status': 'failed'
        })
    
    print(f"\n📊 Progress: {processed + failed}/{len(image_files)} ({processed} ✅, {failed} ❌)")

# Save results to a log file
log_path = os.path.join(DRIVE_OUTPUT_FOLDER, 'generation_log.json')
with open(log_path, 'w') as f:
    json.dump({
        'timestamp': time.strftime('%Y-%m-%d %H:%M:%S'),
        'settings': {
            'prompt': prompt,
            'aspect_ratio': aspect_ratio,
            'duration': duration
        },
        'results': results
    }, f, indent=2)

print("\n🎉 Processing complete!")
print(f"   ✅ Successfully processed: {processed}")
print(f"   ❌ Failed: {failed}")
print(f"\n📁 Videos saved to: {DRIVE_OUTPUT_FOLDER}")
print(f"📝 Log saved to: {log_path}")